In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

In [2]:
EMBEDDING_DIM = 256
HIDDEN_SIZE = 512
BATCH_SIZE = 2
LEARNING_RATE = 0.001
EPOCHS = 10

In [3]:
eng_vocab = {word: i for i, word in enumerate(['<PAD>', '<SOS>', '<EOS>', 'hello', 'world'])}
jap_vocab = {word: i for i, word in enumerate(['<PAD>', '<SOS>', '<EOS>', 'こんにちは', '世界'])}

eng_vocab_size = len(eng_vocab)
jap_vocab_size = len(jap_vocab)

In [4]:
x_data = torch.tensor([[eng_vocab['hello'], eng_vocab['world'], eng_vocab['<EOS>'], eng_vocab['<PAD>']]])
y_data = torch.tensor([[jap_vocab['こんにちは'], jap_vocab['世界'], jap_vocab['<EOS>'], jap_vocab['<PAD>']]])

dataset = TensorDataset(x_data, y_data)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True)
    
    def forward(self, x):
        embedded = self.embedding(x)
        outputs, hidden = self.rnn(embedded)
        return hidden  # Return only the last hidden state

In [6]:
class DecoderRNN(nn.Module):
    def __init__(self, output_size, embedding_dim, hidden_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, hidden):
        embedded = self.embedding(x).unsqueeze(1)  # Add time step dimension
        output, hidden = self.rnn(embedded, hidden)
        output = self.fc(output.squeeze(1))
        return output, hidden

In [7]:
def train(encoder, decoder, dataloader, loss_fn, encoder_optimizer, decoder_optimizer):
    encoder.train()
    decoder.train()
    total_loss = 0
    
    for eng_sentences, jap_sentences in dataloader:
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        hidden = encoder(eng_sentences)  # Get last hidden state from encoder
        decoder_input = torch.tensor([jap_vocab['<SOS>']] * eng_sentences.size(0))  # Fix shape
        loss = 0
        
        for t in range(jap_sentences.size(1)):
            output, hidden = decoder(decoder_input, hidden)
            loss += loss_fn(output, jap_sentences[:, t])
            decoder_input = jap_sentences[:, t]  # Fix shape
        
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

In [12]:
def translate_sentence(encoder, decoder, sentence):
    encoder.eval()
    decoder.eval()
    
    with torch.no_grad():
        input_tensor = torch.tensor([eng_vocab[word] for word in sentence]).unsqueeze(0)
        hidden = encoder(input_tensor)
        decoder_input = torch.tensor([[jap_vocab['<SOS>']]])  # Fix shape
        translated_sentence = []
        
        for _ in range(10):  # Max translation length
            output, hidden = decoder(decoder_input, hidden)
            predicted_token = output.argmax(1).item()
            if predicted_token == jap_vocab['<EOS>']:
                break
            translated_sentence.append(predicted_token)
            decoder_input = torch.tensor([[predicted_token]])  # Fix shape
        
    return [list(jap_vocab.keys())[list(jap_vocab.values()).index(idx)] for idx in translated_sentence]

In [9]:
encoder = EncoderRNN(eng_vocab_size, EMBEDDING_DIM, HIDDEN_SIZE)
decoder = DecoderRNN(jap_vocab_size, EMBEDDING_DIM, HIDDEN_SIZE)

encoder_optimizer = optim.Adam(encoder.parameters(), lr=LEARNING_RATE)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()


In [10]:
for epoch in range(EPOCHS):
    loss = train(encoder, decoder, dataloader, loss_fn, encoder_optimizer, decoder_optimizer)
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

Epoch 1, Loss: 6.3822
Epoch 2, Loss: 0.7919
Epoch 3, Loss: 0.1218
Epoch 4, Loss: 0.0336
Epoch 5, Loss: 0.0138
Epoch 6, Loss: 0.0072
Epoch 7, Loss: 0.0044
Epoch 8, Loss: 0.0030
Epoch 9, Loss: 0.0022
Epoch 10, Loss: 0.0017


In [13]:
sentence = ['hello', 'world', '<EOS>']
translation = translate_sentence(encoder, decoder, sentence)
print("Translated Sentence:", translation)

ValueError: RNN: Expected input to be 2D or 3D, got 4D tensor instead